# Fetching Data from CryptoCompare webpage for latest data 

Mining Calculator Data
Get the price, the total supply, the hash rate, the block reward and the block time. Based on these values you can calculate the profitability of a mining rig.

In [55]:
# import libraries
import requests
from requests import Session
from requests.exceptions import ConnectionError, Timeout, TooManyRedirects
import json
import pandas as pd
from cryptocompare_config import API_Key
import numpy as np
import pymongo
from pymongo import MongoClient
import certifi
from mongo2_config import mongo_uri

In [56]:
api_endpoint = 'https://min-api.cryptocompare.com/data/blockchain/mining/calculator?fsyms=BTC,ETH,XRP,USDT,ADA,XMR,XLM,BUSD,LTC,DOGE&tsyms=USD&api_key='
api_endpoint += API_Key


In [64]:
symbols=['BTC','ETH','XRP','USDT','ADA','XMR','XLM','BUSD','LTC','DOGE']
Top_mining_df = pd.DataFrame()
api_endpoint = 'https://min-api.cryptocompare.com/data/blockchain/mining/calculator?fsyms='
api_endpoint += ','.join(symbols)+"&tsyms=USD&apikey="+API_Key
data_retrieved = requests.get(api_endpoint).json()

for name in symbols:    
    data_retrieved_json = data_retrieved['Data'][name]
    Mining_df = pd.json_normalize(data_retrieved_json)
    Top_mining_df = Top_mining_df.append(Mining_df, ignore_index = True)
    
Top_mining_df

,CoinInfo.Id,CoinInfo.Name,CoinInfo.FullName,CoinInfo.Internal,CoinInfo.ImageUrl,CoinInfo.Url,CoinInfo.NetHashesPerSecond,CoinInfo.BlockNumber,CoinInfo.BlockTime,CoinInfo.BlockReward,CoinInfo.TotalCoinsMined,CoinInfo.AssetLaunchDate,CoinInfo.MaxSupply,Price.USD
0,1182,BTC,Bitcoin,BTC,/media/37746251/btc.png,/coins/btc/overview,203650592238966300000,719680,524.000000,6.250000,1.887391e+07,2009-01-03,2.100000e+07,41627.1500
1,7605,ETH,Ethereum,ETH,/media/37746238/eth.png,/coins/eth/overview,978606978397459.375,14044972,13.248264,2.161901,1.183508e+08,2015-07-30,-1.000000e+00,3110.4400
2,5031,XRP,XRP,XRP,/media/38553096/xrp.png,/coins/xrp/overview,0,69147262,0.000000,0.000000,9.998981e+10,2013-01-01,1.000000e+11,0.7382
3,171986,USDT,Tether,USDT,/media/37746338/usdt.png,/coins/usdt/overview,0,0,0.000000,0.000000,7.832457e+10,2014-10-06,-1.000000e+00,1.0000
4,321992,ADA,Cardano,ADA,/media/37746235/ada.png,/coins/ada/overview,0,6781195,0.000000,0.000000,3.297602e+10,2017-10-05,4.500000e+10,1.3090
5,5038,XMR,Monero,XMR,/media/37746883/xmr.png,/coins/xmr/overview,3239447427,2541623,117.000000,0.738662,1.806607e+07,2014-04-18,-1.000000e+00,195.0300
6,4614,XLM,Stellar,XLM,/media/37746346/xlm.png,/coins/xlm/overview,0,39271830,0.000000,0.000000,5.000180e+10,2014-07-31,-1.000000e+00,0.2403
7,932135,BUSD,BUSD,BUSD,/media/37746248/busd.png,/coins/busd/overview,0,0,0.000000,0.000000,1.433840e+10,2019-09-10,-1.000000e+00,0.9997
8,3808,LTC,Litecoin,LTC,/media/37746243/ltc.png,/coins/ltc/overview,366650691750010,2196805,152.000000,12.522045,6.945811e+07,2011-10-13,8.400000e+07,135.5400
9,4432,DOGE,Dogecoin,DOGE,/media/37746339/doge.png,/coins/doge/overview,416968108670976,4070698,64.000000,10000.000000,1.328795e+11,2013-12-06,-1.000000e+00,0.1590


In [65]:
# Replace NaN as '0'
Top_mining_df.isnull().sum().sum()
Top_mining_df.isna().sum()

CoinInfo.Id                    0
CoinInfo.Name                  0
CoinInfo.FullName              0
CoinInfo.Internal              0
CoinInfo.ImageUrl              0
CoinInfo.Url                   0
CoinInfo.NetHashesPerSecond    0
CoinInfo.BlockNumber           0
CoinInfo.BlockTime             0
CoinInfo.BlockReward           0
CoinInfo.TotalCoinsMined       0
CoinInfo.AssetLaunchDate       0
CoinInfo.MaxSupply             0
Price.USD                      0
dtype: int64

In [66]:
# Drop columns that have null values as those columns are irrelevant
Top_mining_df = Top_mining_df.dropna(axis=1)
Top_mining_df

,CoinInfo.Id,CoinInfo.Name,CoinInfo.FullName,CoinInfo.Internal,CoinInfo.ImageUrl,CoinInfo.Url,CoinInfo.NetHashesPerSecond,CoinInfo.BlockNumber,CoinInfo.BlockTime,CoinInfo.BlockReward,CoinInfo.TotalCoinsMined,CoinInfo.AssetLaunchDate,CoinInfo.MaxSupply,Price.USD
0,1182,BTC,Bitcoin,BTC,/media/37746251/btc.png,/coins/btc/overview,203650592238966300000,719680,524.000000,6.250000,1.887391e+07,2009-01-03,2.100000e+07,41627.1500
1,7605,ETH,Ethereum,ETH,/media/37746238/eth.png,/coins/eth/overview,978606978397459.375,14044972,13.248264,2.161901,1.183508e+08,2015-07-30,-1.000000e+00,3110.4400
2,5031,XRP,XRP,XRP,/media/38553096/xrp.png,/coins/xrp/overview,0,69147262,0.000000,0.000000,9.998981e+10,2013-01-01,1.000000e+11,0.7382
3,171986,USDT,Tether,USDT,/media/37746338/usdt.png,/coins/usdt/overview,0,0,0.000000,0.000000,7.832457e+10,2014-10-06,-1.000000e+00,1.0000
4,321992,ADA,Cardano,ADA,/media/37746235/ada.png,/coins/ada/overview,0,6781195,0.000000,0.000000,3.297602e+10,2017-10-05,4.500000e+10,1.3090
5,5038,XMR,Monero,XMR,/media/37746883/xmr.png,/coins/xmr/overview,3239447427,2541623,117.000000,0.738662,1.806607e+07,2014-04-18,-1.000000e+00,195.0300
6,4614,XLM,Stellar,XLM,/media/37746346/xlm.png,/coins/xlm/overview,0,39271830,0.000000,0.000000,5.000180e+10,2014-07-31,-1.000000e+00,0.2403
7,932135,BUSD,BUSD,BUSD,/media/37746248/busd.png,/coins/busd/overview,0,0,0.000000,0.000000,1.433840e+10,2019-09-10,-1.000000e+00,0.9997
8,3808,LTC,Litecoin,LTC,/media/37746243/ltc.png,/coins/ltc/overview,366650691750010,2196805,152.000000,12.522045,6.945811e+07,2011-10-13,8.400000e+07,135.5400
9,4432,DOGE,Dogecoin,DOGE,/media/37746339/doge.png,/coins/doge/overview,416968108670976,4070698,64.000000,10000.000000,1.328795e+11,2013-12-06,-1.000000e+00,0.1590


In [67]:
# dropping irrelevant columns
Top_mining_df = Top_mining_df.drop(columns=['CoinInfo.ImageUrl', 'CoinInfo.Url', 'CoinInfo.Internal'])
Top_mining_df.head()

,CoinInfo.Id,CoinInfo.Name,CoinInfo.FullName,CoinInfo.NetHashesPerSecond,CoinInfo.BlockNumber,CoinInfo.BlockTime,CoinInfo.BlockReward,CoinInfo.TotalCoinsMined,CoinInfo.AssetLaunchDate,CoinInfo.MaxSupply,Price.USD
0,1182,BTC,Bitcoin,203650592238966300000,719680,524.000000,6.250000,1.887391e+07,2009-01-03,2.100000e+07,41627.1500
1,7605,ETH,Ethereum,978606978397459.375,14044972,13.248264,2.161901,1.183508e+08,2015-07-30,-1.000000e+00,3110.4400
2,5031,XRP,XRP,0,69147262,0.000000,0.000000,9.998981e+10,2013-01-01,1.000000e+11,0.7382
3,171986,USDT,Tether,0,0,0.000000,0.000000,7.832457e+10,2014-10-06,-1.000000e+00,1.0000
4,321992,ADA,Cardano,0,6781195,0.000000,0.000000,3.297602e+10,2017-10-05,4.500000e+10,1.3090


In [68]:
# change column names
Top_mining_df= Top_mining_df.rename(columns={

    'CoinInfo.Name' : 'Symbol',
    'CoinInfo.FullName' : 'Name',
    'CoinInfo.NetHashesPerSecond' : 'NetHashesPerSecond',
    'CoinInfo.BlockNumber': 'BlockNumber',
    'CoinInfo.BlockTime' : 'BlockTime',
    'CoinInfo.BlockReward' : 'BlockReward',
    'CoinInfo.TotalCoinsMined' : 'TotalCoinsMined',
    'CoinInfo.AssetLaunchDate' : 'AssetLaunchDate',
    'CoinInfo.MaxSupply' : 'MaxSupply'})
Top_mining_df

,CoinInfo.Id,Symbol,Name,NetHashesPerSecond,BlockNumber,BlockTime,BlockReward,TotalCoinsMined,AssetLaunchDate,MaxSupply,Price.USD
0,1182,BTC,Bitcoin,203650592238966300000,719680,524.000000,6.250000,1.887391e+07,2009-01-03,2.100000e+07,41627.1500
1,7605,ETH,Ethereum,978606978397459.375,14044972,13.248264,2.161901,1.183508e+08,2015-07-30,-1.000000e+00,3110.4400
2,5031,XRP,XRP,0,69147262,0.000000,0.000000,9.998981e+10,2013-01-01,1.000000e+11,0.7382
3,171986,USDT,Tether,0,0,0.000000,0.000000,7.832457e+10,2014-10-06,-1.000000e+00,1.0000
4,321992,ADA,Cardano,0,6781195,0.000000,0.000000,3.297602e+10,2017-10-05,4.500000e+10,1.3090
5,5038,XMR,Monero,3239447427,2541623,117.000000,0.738662,1.806607e+07,2014-04-18,-1.000000e+00,195.0300
6,4614,XLM,Stellar,0,39271830,0.000000,0.000000,5.000180e+10,2014-07-31,-1.000000e+00,0.2403
7,932135,BUSD,BUSD,0,0,0.000000,0.000000,1.433840e+10,2019-09-10,-1.000000e+00,0.9997
8,3808,LTC,Litecoin,366650691750010,2196805,152.000000,12.522045,6.945811e+07,2011-10-13,8.400000e+07,135.5400
9,4432,DOGE,Dogecoin,416968108670976,4070698,64.000000,10000.000000,1.328795e+11,2013-12-06,-1.000000e+00,0.1590


# Analyzing Top 10 currencies based on Net Hashes Per second

In [70]:
# Top 10 currencies based on Net Hashes per second
Top_mining_df = Top_mining_df.sort_values(by = ["NetHashesPerSecond"], ascending = False)
Top_mining_df.head(20)

,CoinInfo.Id,Symbol,Name,NetHashesPerSecond,BlockNumber,BlockTime,BlockReward,TotalCoinsMined,AssetLaunchDate,MaxSupply,Price.USD
0,1182,BTC,Bitcoin,203650592238966300000,719680,524.000000,6.250000,1.887391e+07,2009-01-03,2.100000e+07,41627.1500
1,7605,ETH,Ethereum,978606978397459.375,14044972,13.248264,2.161901,1.183508e+08,2015-07-30,-1.000000e+00,3110.4400
9,4432,DOGE,Dogecoin,416968108670976,4070698,64.000000,10000.000000,1.328795e+11,2013-12-06,-1.000000e+00,0.1590
8,3808,LTC,Litecoin,366650691750010,2196805,152.000000,12.522045,6.945811e+07,2011-10-13,8.400000e+07,135.5400
5,5038,XMR,Monero,3239447427,2541623,117.000000,0.738662,1.806607e+07,2014-04-18,-1.000000e+00,195.0300
2,5031,XRP,XRP,0,69147262,0.000000,0.000000,9.998981e+10,2013-01-01,1.000000e+11,0.7382
3,171986,USDT,Tether,0,0,0.000000,0.000000,7.832457e+10,2014-10-06,-1.000000e+00,1.0000
4,321992,ADA,Cardano,0,6781195,0.000000,0.000000,3.297602e+10,2017-10-05,4.500000e+10,1.3090
6,4614,XLM,Stellar,0,39271830,0.000000,0.000000,5.000180e+10,2014-07-31,-1.000000e+00,0.2403
7,932135,BUSD,BUSD,0,0,0.000000,0.000000,1.433840e+10,2019-09-10,-1.000000e+00,0.9997


# Exporting data to csv and Uploading on MongoDb

In [72]:
# exporting the dataframe to csv
Top_mining_df.to_csv('Mining_NetHashes_cryptocompare_data.csv')

In [73]:
# Read in mongodb server location as client
client = MongoClient(mongo_uri,tlsCAFile=certifi.where())

In [74]:
# Find Databases
client.list_database_names()

['Kaggle_data', 'Latest_Cryptodata', 'admin', 'local']

In [75]:
# connecting to the db on MongoDb and creating collection in the db
db = client['Latest_Cryptodata']
collection = db['Mining_NetHashes_cryptocompare_data']

In [76]:
Top_mining_df.reset_index(inplace=True)
Top_mining_df_dict = Top_mining_df.to_dict("records")

In [77]:
# Insert collection
collection.insert_many(Top_mining_df_dict)

OverflowError: MongoDB can only handle up to 8-byte ints